## 1. 웹 페이지 요청하기

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from typing import List, Dict

def cultural_heritage_info():
    
    url = "https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EB%B3%B4%EB%AC%BC_(2020%EB%85%84%EB%8C%80_%EC%A0%84%EB%B0%98)"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # 오류가 있으면 예외를 발생시킴
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"요청 중 에러 발생: {e}")
        return ""

# 테스트: 첫 페이지 가져오기
html = cultural_heritage_info()
print("HTML 일부:", html[:500])  # 처음 500자만 출력

HTML 일부: <!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


## 2. HTML 파싱하기

In [ ]:
def Heritage_info(html: str) -> List[Dict]:
    
    stores = []
    soup = BeautifulSoup(html, 'html.parser')
    # 모든 tbody를 가져옴
    tbodies = soup.find_all('tbody')
    # 마지막 tbody를 제외한 나머지 tbody를 반복
    for tbody in tbodies[:-1]:  # 마지막 tbody를 제외
        # tbody 내의 모든 tr 요소 찾기
        for tr in tbody.find_all("tr"):
            # 각 tr 내의 td와 th 요소 찾기
            tds = tr.find_all('td')
            ths = tr.find_all('th')
            if len(ths) < 1:
                continue
            if len(tds) < 6:
                continue
            store = {
                'number': ths[0].text.strip(),
                'designation': tds[1].text.strip(),
                'location': tds[2].text.strip(),
            }
            # 특정 조건에 따라 데이터 필터링
            if "문화유산(시도별·시군구별)" in store['number']:
                continue  # 특정 값이 포함된 경우 건너뛰기
            stores.append(store)
    return stores  # 모든 tbody에서 수집한 데이터 반환
# 테스트: 첫 페이지 파싱하기
stores = Heritage_info(html)
print(f"찾은 문화 유산 수: {len(stores)}")
print("\n첫 번째 문화 유산 정보:")
print(json.dumps(stores[0], ensure_ascii=False, indent=2))  # 첫 번째 매장 정보 출력
print("\n 마지막 문화 유산 정보:")
print(json.dumps(stores[-1], ensure_ascii=False, indent=2))  # 첫 번째 매장 정보 출력

찾은 문화 유산 수: 104

첫 번째 문화 유산 정보:
{
  "number": "1219-4호",
  "designation": "대방광원각수다라요의경(언해) 권상1의2(大方廣圓覺脩多羅了義經(諺解) 卷上一之二)",
  "location": "서울 동대문구"
}

 마지막 문화 유산 정보:
{
  "number": "-",
  "designation": "거제 기성관(巨濟 岐城館)",
  "location": "경상남도 거제시"
}


## 3. 데이터 저장하기

In [ ]:
def save_to_files(stores: List[Dict], base_path: str = "./"):
    
    # CSV 파일로 저장
    df = pd.DataFrame(stores)
    csv_path = f"{base_path}Cultural_Heritage_info13.csv"
    df.to_csv(csv_path, encoding='utf-8', index=False)
    
    
    print(f"CSV 파일 저장 완료: {csv_path}")
    
    # 데이터 미리보기
    print("\n데이터 미리보기:")
    print(df.head())

# 테스트: 첫 페이지 데이터 저장하기
save_to_files(stores)

CSV 파일 저장 완료: ./Cultural_Heritage_info12.csv

데이터 미리보기:
    number                                        designation  \
0  1219-3호  대방광원각수다라요의경(언해) 권하 1의1∼2, 2의1∼2(大方廣圓覺脩多羅了義經(諺解...   
1  1281-4호  자치통감 권226~229(資治通鑑 卷二百二十六～二百二十九)(Jachi tonggam...   
2    1851호               논산 쌍계사 소조석가여래삼불좌상(論山 雙溪寺 塑造釋迦如來三佛坐像)   
3    1852호  남원 선원사 목조지장보살삼존상 및 소조시왕상 일괄(南原 禪院寺 木造地藏菩薩三尊像 및...   
4    1853호             완주 정수사 목조아미타여래삼존좌상(完州 淨水寺 木造阿彌陀如來三尊坐像)   

              location  
0   부산 기장군 일광면 용천리 취장사  
1  울산 남구 두왕로 277 울산박물관  
2     충남 논산시 양촌면 중산리 3  
3       전북 남원시 도통동 392  
4   전북 완주군 상관면 마치리 137  


## 4. 전체 과정 실행하기

In [ ]:
def main():
    stores = []
    
    # 매장 정보 수집
    print("문화 유산 정보 수집 중...")

    html = cultural_heritage_info()
    if html:
        page_stores = Heritage_info(html)
        stores.extend(page_stores)
        print(f"{len(page_stores)}개의 문화 유산 수집 완료")
    
    if not stores:
        print("문화재 정보를 가져오는데 실패했습니다.")
        return
        
    print(f"\n총 {len(stores)}개의 문화 유산을 수집했습니다.")

    # 데이터 저장
    try:
        save_to_files(stores)
    except Exception as e:
        print(f"데이터 저장 중 에러 발생: {e}")

# 전체 과정 실행
main()

문화 유산 정보 수집 중...
100개의 문화 유산 수집 완료

총 100개의 문화 유산을 수집했습니다.
CSV 파일 저장 완료: ./Cultural_Heritage_info4.csv

데이터 미리보기:
   number                                        designation  \
0    583호  전주 풍패지관(全州 豊沛之館)(Pungpaejigwan Guesthouse, Jeo...   
1    584호  구례 윤문효공 신도비(求禮 尹文孝公 神道碑)(Stele for Yun Munhyog...   
2    585호  퇴우이선생진적(退尤二先生眞蹟)(Manuscript of Yi Hwang and So...   
3  586-1호  이언적 수고본 일괄-속대학혹문(李彦迪 手稿本 一括-續大學或問)(Manuscripts...   
4  586-2호  이언적 수고본 일괄-대학장구보유(李彦迪 手稿本 一括-大學章句補遺)(Manuscrip...   

                            location  
0                 전북 전주시 완산구 중앙동 3가1  
1               전남 구례군 산동면 이평리 산91-1  
2                             서울 용산구  
3              경북 경주시 안강읍 옥산리 7 옥산서원  
4  경북 경주시 안강읍 옥산서원길 300-3, 독락당 (옥산리)  


In [ ]:
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# 두 데이터프레임을 합칩니다.
merged_df = pd.concat([df1, df2], ignore_index=True)

# 합쳐진 데이터프레임을 새로운 CSV 파일로 저장합니다.
merged_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"합쳐진 CSV 파일 저장 완료: {output_file}")